# 12 與 14 的結合版本

In [1]:
import pymysql
import numpy as np
import pandas as pd
import time,os,ccxt
from datetime import datetime,timedelta
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import talib as ta

pd.options.display.float_format = '{:.2f}'.format
# pd.options.mode.chained_assignment = None  # default='warn'

def read_from_mysql(tablename):
    conn = pymysql.connect(host='192.168.68.11',user='testuser',passwd='test1234',db='testuser',charset='utf8')


    sql="select * from "
    sql=sql +tablename 
    

    df= pd.read_sql(sql, conn)
    conn.close()

    return df

def RSI_up_cross(df):
    if df.pre_RSI_rank==0 and df.RSI_rank==1 :
        return 1
    elif df.pre_RSI_rank==1 and df.RSI_rank==0 :
        return -1
    else:
        return 0


    

def handle_data(df,RSI_short,factor_vol,factor_out):  #factor_in: 1.6~8 , factor_out: 0.35~0.99
    # 把string改成float，以利後續計算：
    df['price_open']=df['price_open'].astype(float)
    df['price_high']=df['price_high'].astype(float)
    df['price_low']=df['price_low'].astype(float)
    df['price_close']=df['price_close'].astype(float)
    df['volume']=df['volume'].astype(float)

    #調整close的值：
    from scipy.stats import hmean
    df['adj_close']=df[['price_open','price_high','price_low','price_close']].apply(hmean,axis=1) #調和平均數



    #集中刪除區：
    df['RSI_short'] = ta.RSI(df['adj_close'].values,timeperiod=RSI_short)
    df['RSI_long'] = ta.RSI(df['adj_close'].values,timeperiod=100)   
    df['DEMA_200'] = ta.DEMA(df['adj_close'].values,timeperiod=200)

    df['vol_avg']=df['volume'].rolling(66).mean()
    
    df.dropna(inplace=True)





    #--------condition1(RSI):--------------------------------
    # df['RSI_short'] = ta.RSI(df['adj_close'].values,timeperiod=RSI_short)
    # df['RSI_long'] = ta.RSI(df['adj_close'].values,timeperiod=RSI_long)

    df['RSI_rank']=(df['RSI_short']-df['RSI_long']).apply(lambda x:1 if x>0 else 0)
    df['pre_RSI_rank']=df['RSI_rank'].shift(1)    
    df.dropna(inplace=True)
    df['RSI_rank_up_cross']=df.apply(RSI_up_cross,axis=1) #1:金叉 ，-1:死叉  0:其它


    #--------condition2(量有上來):--------------------------------
    # df['vol_avg']=df['volume'].rolling(66).mean()
    df['vol_rank']=(df['volume']/(factor_vol*df['vol_avg'])).apply(lambda x:1 if x>1 else 0)

 

    #---------condition3:------------------------------
    df['ATR'] = ta.ATR(df['price_high'].values,df['price_low'].values,df['adj_close'].values,timeperiod=66)
    
    df['HH']=df['adj_close'].rolling(66).max()
    df['Chandelier_bull_rank']=(df['adj_close']-(df['HH']-factor_out*df['ATR'])).apply(lambda x:1 if x>0 else 0) #1 表示收盤價在吊燈上方，繼續持有

    df['LL']=df['adj_close'].rolling(66).min()
    df['Chandelier_bear_rank']=(df['adj_close']-(df['LL']+factor_out*df['ATR'])).apply(lambda x:-1 if x<0 else 0) #-1 表示收盤價在吊燈下方，繼續持有空單


    #--------condition4 (外加1h 保護也會用到)--------------------------------
    # df['DEMA_200'] = ta.DEMA(df['adj_close'].values,timeperiod=200)
    df['DEMA_200_rank']=(df['adj_close']-df['DEMA_200']).apply(lambda x:1 if x>0 else -1)




    #定義進出訊號：(金叉進，吊燈出)
    df['entries']=(df['RSI_rank_up_cross']+df['vol_rank']+df['Chandelier_bull_rank']).apply(lambda x:1 if x==3  else 0) 
    df['entries_bear']=(df['RSI_rank_up_cross']+df['Chandelier_bear_rank']+df['DEMA_200_rank']).apply(lambda x:1 if x==-3  else 0) #3個都負值就開空單(即1就開空單)
    df['exits']=(df['Chandelier_bull_rank']).apply(lambda x:1 if x==0 else 0) 
    df['exits_bear']=(df['Chandelier_bear_rank']).apply(lambda x:1 if x==0 else 0) 


    return df


# GA

In [2]:
#GA combine test:
strategy='VR-1'
symbols=['ADA/USDT','BCH/USDT','DOGE/USDT','DOT/USDT','FTM/USDT','LINK/USDT','LTC/USDT','MATIC/USDT','SOL/USDT','TRX/USDT','XRP/USDT','BTC/USDT','ETH/USDT']
# symbols=['DOT/USDT','MATIC/USDT','TRX/USDT']

for symbol in symbols:


    df_raw=read_from_mysql(symbol.replace('/','_')+"_15m")
    print(f'{symbol} data length: {len(df_raw)}')

    def train_GA(x,y,z):
        #讀取資料：
        # symbol='DOGE/USDT'
        # df=read_from_mysql(symbol.replace('/','_')+"_15m")
        df=df_raw.copy()
        # df=df.iloc[-99999:-9999,:] #留下-9999當測試普適性用的
        df=df.iloc[-33333:,:] #這幾個交易對都是適合的，直接用最近的33333筆資料算最佳參數
        
        
        ########## parameters #######################

        RSI_short=x
        factor_vol=y/10  #factor_in: 1.2~4.4 (2^5) 
        factor_out=z/10 #factor_out: 5~37 (2^5)

        ############## handle_data ##############################
        df=handle_data(df,RSI_short,factor_vol,factor_out)

        
        ############
        import vectorbt as vbt #看來都要塞array進去當參數，用 df.values
        entries=df.apply(lambda x: (x['entries']==1),axis=1)
        exits=df.apply(lambda x: (x['exits']==1),axis=1)


        pf=vbt.Portfolio.from_signals(df['adj_close'].values,entries=entries,exits=exits,init_cash=100,fees=0.0004) 
        # result=pf.stats()['Total Return [%]']
        # result=pf.stats()['Win Rate [%]'] #改用勝率來看這種方法好不好，用total return怕over fitting
        result=pf.stats()['Avg Winning Trade [%]'] + pf.stats()['Avg Losing Trade [%]']*2
      

        # if pf.stats()['Total Return [%]']> pf.stats()['Benchmark Return [%]'] :
        if  pf.stats()['Total Fees Paid']/pf.stats()['Total Return [%]'] <0.3 and \
            pf.stats()['Total Return [%]']> (pf.stats()['Benchmark Return [%]']*1.5 ) and \
            pf.stats()['Profit Factor']>1.2  and \
            pf.stats()['Total Trades']>33 and  \
            pf.stats()['Win Rate [%]']>33 and  \
            pf.stats()['Max Drawdown [%]']<21 : 
          

            return result
        else:
            return 0.01
        



    # if not (np.isnan(prob[j])):
    #     outputText += np.random.choice(data, p=prob)

        
        
        
    def F(pop):
        
        
        
        scores=[]
        for k in range(len(pop)):
            
            #將DNA拆成3個值
            #res = int("".join(str(x) for x in pop[0][0:3]), 2)
            x=int("".join(str(x) for x in pop[k][0:5]), 2)+5 #5~37
            y=int("".join(str(x) for x in pop[k][5:11]), 2)+11 #11~75 
            z=int("".join(str(x) for x in pop[k][11:16]), 2)+11 #11~43    (/10)     


    #         if (long_period>middle_period and middle_period>short_period): #舊的寫法要判斷長天期MA要> 小天期MA
            if 1==1:
                
                score=train_GA(x,y,z)
                scores.append(score)

                
            
            else:
    #             print('xxxxxxxxxxxxxxxxxxx  wrong number setup  xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
                score=0.1
                scores.append(score)
                continue
            
    
            
            
        return np.array(scores)


    # GA testing:

    # import numpy as np
    # import matplotlib.pyplot as plt

    DNA_SIZE = 16            # DNA length (上面parameters的總合)
    POP_SIZE = 999           # population size
    CROSS_RATE = 0.9         # mating probability (DNA crossover)
    MUTATION_RATE = 0.009    # mutation probability
    N_GENERATIONS = 3
    X_BOUND = [3,39]         # x upper and lower bounds (這個沒用到了！)


    # def F(x): return np.sin(10*x)*x + np.cos(2*x)*x     # to find the maximum of this function


    # find non-zero fitness for selection
    def get_fitness(pred): return pred + 1e-3 - np.min(pred) 


    # convert binary DNA to decimal and normalize it to a range(0, 5)
    def translateDNA(pop): 
    #     return pop.dot(2 ** np.arange(DNA_SIZE)[::-1]) / float(2**DNA_SIZE-1) * X_BOUND[1]
        return (pop.dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1) * X_BOUND[1]).astype(int)+X_BOUND[0]




    def select(pop, fitness):    # nature selection wrt pop's fitness
        idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True,
                            p=fitness/fitness.sum())
        return pop[idx]


    def crossover(parent, pop):     # mating process (genes crossover)
        if np.random.rand() < CROSS_RATE:
            i_ = np.random.randint(0, POP_SIZE, size=1)                             # select another individual from pop
            cross_points = np.random.randint(0, 2, size=DNA_SIZE).astype(np.bool)   # choose crossover points
            parent[cross_points] = pop[i_, cross_points]                            # mating and produce one child
        return parent


    def mutate(child):
        for point in range(DNA_SIZE):
            if np.random.rand() < MUTATION_RATE:
                child[point] = 1 if child[point] == 0 else 0
        return child


    pop = np.random.randint(2, size=(POP_SIZE, DNA_SIZE))   # initialize the pop DNA

    # plt.ion()       # something about plotting
    # x = np.linspace(*X_BOUND, 200)
    # plt.plot(x, F(x))


    for iii in range(N_GENERATIONS):
        F_values = F(pop)    # compute function value by extracting DNA

        # something about plotting
    #     if 'sca' in globals(): sca.remove()
    #     sca = plt.scatter(translateDNA(pop), F_values, s=200, lw=0, c='red', alpha=0.5); plt.pause(0.05)

        # GA part (evolution)
        fitness = get_fitness(F_values)
        x_result=int("".join(str(x) for x in pop[np.argmax(fitness)][0:5]), 2)+5
        y_result=round((int("".join(str(x) for x in pop[np.argmax(fitness)][5:11]), 2)+11)/10,1)
        z_result=round((int("".join(str(x) for x in pop[np.argmax(fitness)][11:16]), 2)+11)/10,1)
        final_result=np.max(F_values)

        print(f'G{iii} --Most fitted DNA:  {pop[np.argmax(fitness)]} --> {x_result},{y_result},{z_result}---> {final_result}')
        


        #2023/11/30 add 寫入結果到GA.csv :
        df=pd.read_csv('GA-15.csv')
        mydate=datetime.now().strftime("%Y%m%d")
        df.loc[len(df.index)]=[mydate,strategy,symbol,x_result,y_result,z_result,final_result]
        # df.iloc[-1,0]=strategy
        # df.iloc[-1,1]=symbol
        # df.iloc[-1,2]= x_result
        # df.iloc[-1,3]= y_result
        # df.iloc[-1,4]= z_result
        # df.iloc[-1,5]= final_result

        df.to_csv('GA-15.csv',index=False)



        # 2022/09/10 增加突變率變化：(開始時突變率高)
        if iii<3:
            MUTATION_RATE = 0.09
        else:
            MUTATION_RATE = 0.003
            
        
        
        pop = select(pop, fitness)
        pop_copy = pop.copy()
        for parent in pop:
            child = crossover(parent, pop_copy)
            child = mutate(child)
            parent[:] = child       # parent is replaced by its child
        
    

ADA/USDT data length: 198208
G0 --Most fitted DNA:  [1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0] --> 22,1.6,3.3---> 2.2993568209673994
G1 --Most fitted DNA:  [1 0 0 1 0 0 0 0 1 0 1 1 0 1 0 1] --> 23,1.6,3.2---> 2.5957758837123865
G2 --Most fitted DNA:  [1 0 0 1 0 0 0 0 1 0 1 1 0 1 0 1] --> 23,1.6,3.2---> 2.5957758837123865
BCH/USDT data length: 141780
G0 --Most fitted DNA:  [0 0 0 0 1 1 0 0 1 1 0 1 1 1 1 0] --> 6,4.9,4.1---> 0.01
G1 --Most fitted DNA:  [1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0] --> 21,4.3,1.9---> 0.01
G2 --Most fitted DNA:  [1 0 1 0 0 0 1 0 1 1 0 1 0 1 0 1] --> 25,3.3,3.2---> 0.01
DOGE/USDT data length: 155739
G0 --Most fitted DNA:  [0 0 0 1 1 0 1 0 1 1 1 1 0 0 1 0] --> 8,3.4,2.9---> 2.347738957199721
G1 --Most fitted DNA:  [0 0 0 1 1 0 1 1 0 0 1 1 0 0 0 1] --> 8,3.6,2.8---> 2.9281454023340947
G2 --Most fitted DNA:  [0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 0] --> 15,3.5,2.5---> 2.2510053764335423
DOT/USDT data length: 116443
G0 --Most fitted DNA:  [1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1] --> 35,2.0,3.6---

### 回測：

In [5]:
symbol='DOT/USDT'
df_raw=read_from_mysql(symbol.replace('/','_')+"_15m")
print(len(df_raw))

115771


In [7]:
#讀取資料：
# symbol='BCH/USDT'
# df=read_from_mysql(symbol.replace('/','_')+"_15m")
df=df_raw.copy()
# df=df.iloc[-99999:-9999,:] #all(train+Test), for best
df=df.iloc[-9999:,:] #for 普適性33
########## parameters #######################

RSI_short=5
rolling_window=4
factor_out=1.9

############## handle_data ##############################
df=handle_data(df,RSI_short,rolling_window,factor_out)
# df.head()

import vectorbt as vbt #看來都要塞array進去當參數，用 df.values

entries=df.apply(lambda x: (x['entries']==1),axis=1)
exits=df.apply(lambda x: (x['exits']==1),axis=1)
# print(entries.value_counts(),'___',exits.value_counts()) #計算true/false個數
pf=vbt.Portfolio.from_signals(df['adj_close'].values,entries=entries,exits=exits,init_cash=100,fees=0.0004) 
print(pf.stats())

Start                        106171.00
End                          115770.00
Period                         9600.00
Start Value                     100.00
End Value                       172.32
Total Return [%]                 72.32
Benchmark Return [%]             47.38
Max Gross Exposure [%]          100.00
Total Fees Paid                   2.30
Max Drawdown [%]                 12.89
Max Drawdown Duration          3535.00
Total Trades                     27.00
Total Closed Trades              26.00
Total Open Trades                 1.00
Open Trade PnL                   10.21
Win Rate [%]                     46.15
Best Trade [%]                   17.73
Worst Trade [%]                  -3.59
Avg Winning Trade [%]             5.99
Avg Losing Trade [%]             -1.38
Avg Winning Trade Duration      388.58
Avg Losing Trade Duration       184.07
Profit Factor                     4.09
Expectancy                        2.39
dtype: float64


In [8]:
#draw chart:
pf.orders.plot(width=1500)

FigureWidget({
    'data': [{'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showl…

### 每周自動update GA parameters

In [5]:
df_para=pd.read_csv('GA-14.csv')
df_para.head()

,date,strategy,symbol,x,y,z,result
0,20231212,RSI-6,DOT/USDT,32,14,39,1.31
1,20231212,RSI-6,DOT/USDT,31,13,39,3.02
2,20231212,RSI-6,DOT/USDT,31,13,39,3.02
3,20231212,RSI-6,MATIC/USDT,5,15,40,0.24
4,20231212,RSI-6,MATIC/USDT,5,15,39,0.86


In [17]:
mask=(df_para['strategy']=='RSI-6') & (df_para['symbol']=='DOT/USDT') & (df_para['result']!=0.01)   #每個條件要()起來，用&相連接
print(df_para[mask])
print(df_para[mask]['x'].iloc[-1])

        date strategy    symbol   x   y   z  result
0   20231212    RSI-6  DOT/USDT  32  14  39    1.31
1   20231212    RSI-6  DOT/USDT  31  13  39    3.02
2   20231212    RSI-6  DOT/USDT  31  13  39    3.02
18  20231212    RSI-6  DOT/USDT  33  10  34    1.91
19  20231212    RSI-6  DOT/USDT  33  10  32    2.63
20  20231212    RSI-6  DOT/USDT  33  10  32    2.63
33


### 實戰勝率：

In [3]:
#計算勝利及總獲利：

import pandas as pd

for coin in ['ADA','BCH','BTC','DOGE','ETH','FTM','MATIC','XRP','SOL','LINK','ADA','TRX','DOT','LTC','SHIB']:
# for coin in ['ADA','DOGE','DOT','LTC','MATIC','XRP']:

    df=pd.read_csv('./trading/'+coin+'USDT.csv')
    df.dropna(inplace=True)

    print(f'-----------------{coin}---------------------------------------')
    
    # strategys=df['strategy'].unique().tolist()
    strategys=['VR-1']
    
    for strategy in strategys:
    
        
        df2=df[df['strategy']==strategy]

        trade_cnt=len(df2)
        win_rate=round(df2['balance'].agg(lambda x: x>0).sum() /len(df2),2)
        benefit=round(df2['balance'].sum(),1)
        benefit2=round(df2['balance'].sum()-trade_cnt*2*0.04,1)
        
        print(f'{strategy} 交易次數：{trade_cnt} ,  勝率：{win_rate}  ,  獲利：{benefit} , 實質獲利：{benefit2}')


-----------------ADA---------------------------------------
VR-1 交易次數：3 ,  勝率：0.33  ,  獲利：0.9 , 實質獲利：0.7
-----------------BCH---------------------------------------
VR-1 交易次數：0 ,  勝率：nan  ,  獲利：0.0 , 實質獲利：0.0
-----------------BTC---------------------------------------
VR-1 交易次數：0 ,  勝率：nan  ,  獲利：0.0 , 實質獲利：0.0
-----------------DOGE---------------------------------------
VR-1 交易次數：0 ,  勝率：nan  ,  獲利：0.0 , 實質獲利：0.0
-----------------ETH---------------------------------------
VR-1 交易次數：0 ,  勝率：nan  ,  獲利：0.0 , 實質獲利：0.0
-----------------FTM---------------------------------------
VR-1 交易次數：1 ,  勝率：0.0  ,  獲利：-1.0 , 實質獲利：-1.1
-----------------MATIC---------------------------------------
VR-1 交易次數：4 ,  勝率：0.0  ,  獲利：-2.5 , 實質獲利：-2.8
-----------------XRP---------------------------------------
VR-1 交易次數：3 ,  勝率：0.33  ,  獲利：2.4 , 實質獲利：2.2
-----------------SOL---------------------------------------
VR-1 交易次數：0 ,  勝率：nan  ,  獲利：0.0 , 實質獲利：0.0
-----------------LINK----------------------------------

C:\Users\derek\AppData\Local\Temp\ipykernel_5452\3464321870.py:22: RuntimeWarning: invalid value encountered in double_scalars
  win_rate=round(df2['balance'].agg(lambda x: x>0).sum() /len(df2),2)
C:\Users\derek\AppData\Local\Temp\ipykernel_5452\3464321870.py:22: RuntimeWarning: invalid value encountered in double_scalars
  win_rate=round(df2['balance'].agg(lambda x: x>0).sum() /len(df2),2)
C:\Users\derek\AppData\Local\Temp\ipykernel_5452\3464321870.py:22: RuntimeWarning: invalid value encountered in double_scalars
  win_rate=round(df2['balance'].agg(lambda x: x>0).sum() /len(df2),2)
C:\Users\derek\AppData\Local\Temp\ipykernel_5452\3464321870.py:22: RuntimeWarning: invalid value encountered in double_scalars
  win_rate=round(df2['balance'].agg(lambda x: x>0).sum() /len(df2),2)
C:\Users\derek\AppData\Local\Temp\ipykernel_5452\3464321870.py:22: RuntimeWarning: invalid value encountered in double_scalars
  win_rate=round(df2['balance'].agg(lambda x: x>0).sum() /len(df2),2)


ZeroDivisionError: division by zero

In [2]:
!pip install python-binance -U

Defaulting to user installation because normal site-packages is not writeable
